In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import os
from sklearn.externals import joblib
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

E:\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [124]:
feature_selected_list = ['DE_time_mean',
 'DE_time_std',
 'DE_time_max',
 'DE_time_min',
 'DE_time_rms',
 'DE_time_ptp',
 'DE_time_median',
 'DE_time_iqr',
 'DE_time_pr',
 'DE_time_skew',
 'DE_time_kurtosis',
 'DE_time_var',
 'DE_time_amp',
 'DE_time_smr',
 'DE_time_wavefactor',
 'DE_time_peakfactor',
 'DE_time_pulse',
 'DE_time_margin',
 'DE_freq_mean',
 'DE_freq_std',
 'DE_freq_max',
 'DE_freq_min',
 'DE_freq_rms',
 'DE_freq_median',
 'DE_freq_iqr',
 'DE_freq_pr',
# 'DE_freq_f2',
# 'DE_freq_f3',
# 'DE_freq_f4',
 'DE_freq_f5',
 'DE_freq_f6',
 'DE_freq_f7',
 'DE_freq_f8',
 'label']

做出训练数据和测试数据

In [134]:
#feature_all = pd.read_csv('../Data_set/feature_4/feature_all.csv')
feature_all = pd.read_csv('../Data_set/feature_4_new/feature_all.csv')
train_data = feature_all[feature_selected_list]
#feature_selected.to_csv('../Data_set/feature_4/feature_selected.csv',index=False)

#path = '../Data_set/feature_test_all'
path = '../Data_set/feature_test_all_new'
files = os.listdir(path)
index = 0
for file2select in files:
    index = index + 1
    dffile = pd.read_csv(path + '/' + file2select)
    #特征筛选
    dffile = dffile[feature_selected_list[:-1]]
    dffile.to_csv('../Data_set/feature_test_new/TEST'+str(index).zfill(2)+'.csv',index=False)  

跑模型

In [135]:
# 模型初始化，设置random_state保证可复现性，便于观察优化
train_data_y = train_data['label']
# 除去标签的所有列就是特征
train_data_x = train_data.drop(['label'], axis=1)

model_lgb_default = lgb.LGBMClassifier(random_state=2019)
# 模型训练
model_lgb_default.fit(train_data_x, train_data_y)
#joblib.dump(model_lgb_default, './model/lightgbm1_2.model')                 #要改的地方

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=2019, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

测试模型

In [153]:
model = model_lgb_default

#测试集文件路径
path = '../Data_set/feature_test_new'
files = os.listdir(path)
index = 0

result = pd.DataFrame()

for test_file in files:
    predict_res = pd.DataFrame()
    index = index + 1 
    dftest_file = pd.read_csv(path + '/' + test_file)
    predict = model.predict(dftest_file)
    predict_res['label'] = predict
    predict_res['filename'] = 'TEST'+str(index).zfill(2)
    result = pd.concat([result,predict_res])

result.to_csv('../Data_set/result/result1_3.csv',index=False)                 #要改的地方2

打分

In [154]:
#res = pd.read_csv('../Data_set/result/result1_2.csv')                            #要改的地方3

switch ={'TEST01':1,
         'TEST02':1,
         'TEST03':3,   
         'TEST04':2,
         'TEST05':2,  
         'TEST06':2,
         'TEST07':2,  
         'TEST08':0,
         'TEST09':2,
         'TEST10':2,    
         'TEST11':2,
         'TEST12':3,
         'TEST13':3,
         'TEST14':1,}

y_true = []
for ans in result['filename']:
    y_true.append(switch.get(ans))

y_pred = list(result['label'])

#y_pred, y_true =[1,0,1,0], [0,0,1,0]
Answer = classification_report(y_true=y_true, y_pred=y_pred, digits=3, output_dict = True)
#Answer['0']['f1-score']

score = (0.3*(Answer['1']['f1-score']+Answer['2']['f1-score']+Answer['3']['f1-score'])+0.1*Answer['0']['f1-score'])*100
score

90.95460550691587